In [8]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
batch_size=32
size=64,64
train_datagen = ImageDataGenerator(validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    "/content/drive/My Drive/Photos",
    target_size=(224,224),
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    "/content/drive/My Drive/Photos", 
    target_size=(224,224),
    subset='validation') 

Found 320 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [3]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
from keras.applications import resnet50
import tensorflow as tf

img_rows, img_cols, img_channel = 224, 224, 3


#base_model = resnet50.ResNet50(weights='imagenet',input_shape=(img_rows, img_cols, img_channel))

base_model=tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None
)

In [5]:

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [6]:

def init_callbacks():
  from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
  base_path = "/content/drive/My Drive/weights/ResNet/ResNet2/"

 
  trained_models_path = base_path + 'model_weights'
  model_names = trained_models_path + '.{epoch:04d}--{val_loss:.4f}--{val_accuracy:.4f}.h5'
  model_checkpoint = ModelCheckpoint(model_names, monitor = 'val_accuracy', verbose=1,save_best_only=True)

  callbacks = [model_checkpoint]
  return callbacks

In [7]:

hist = model.fit_generator(steps_per_epoch=train_generator.samples // batch_size, generator=train_generator, validation_data= validation_generator, validation_steps=validation_generator.samples // batch_size,epochs=25,callbacks= init_callbacks())

model.summary()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
10/10 [==============================] - 266s 25s/step - loss: 0.6915 - accuracy: 0.6838 - val_loss: 0.6891 - val_accuracy: 0.7500

Epoch 00001: val_accuracy improved from -inf to 0.75000, saving model to /content/drive/My Drive/weights/ResNet/ResNet2/model_weights.0001--0.6891--0.7500.h5
Epoch 2/25
10/10 [==============================] - 259s 25s/step - loss: 0.6910 - accuracy: 0.7234 - val_loss: 0.6896 - val_accuracy: 0.7500

Epoch 00002: val_accuracy did not improve from 0.75000
Epoch 3/25
10/10 [==============================] - 257s 25s/step - loss: 0.6904 - accuracy: 0.7325 - val_loss: 0.6891 - val_accuracy: 0.7500

Epoch 00003: val_accuracy did not improve from 0.75000
Epoch 4/25
10/10 [==============================] - 270s 26s/step - loss: 0.6892 - accuracy: 0.7491 - val_loss: 0.6885 - val_accuracy: 0.7500

Epoch 00004: val_accuracy did not improve from 0.75000
Epoch 5/25
10/10 [==============================] - 261s 26s/step - loss: 0.6884 - accuracy: 0.7494 - val